In [13]:
import pickle
import numpy as np
import pandas as pd
import glob
import nltk
import unidecode
import contractions
from nltk.corpus import stopwords

with open('partial_fit_model_no_stop', 'rb') as files:
    l = pickle.load(files)
    
with open('count_vectorizer_no_stop', 'rb') as f:
    t = pickle.load(f)    

with open('label_encoder_no_stop', 'rb') as fil:
    v = pickle.load(fil)

In [ ]:
files = []
for file in glob.glob('*.csv'):
    files.append(file)

In [ ]:
print(f'''Number of .csv files: {len(files)}''')

In [4]:
l.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64)

In [32]:
stop = stopwords.words('english')

In [33]:
def expand_contractions(text):
    expanded_text = contractions.fix(text) 
    return expanded_text

In [34]:
contracted_stop = []
for i in stop:
    contracted_stop.append(expand_contractions(i))

In [35]:
contracted_stw = []
for i in contracted_stop:
    contracted_stw.append(i)

In [36]:
all_stops = stop + contracted_stw

In [37]:
len(all_stops) 

358

In [38]:
from bs4 import BeautifulSoup
import requests

In [39]:
page = requests.get('https://www.ef-australia.com.au/english-resources/english-vocabulary/top-3000-words/')
soup = BeautifulSoup(page.text, 'lxml')

In [40]:
eng_words = []
for i in soup.findAll('p')[11:]:
    for j in i:
        if j!=' ' and j!='':
            eng_words.append(j.text)

In [41]:
eng_words = eng_words + all_stops

In [42]:
len(eng_words)

6357

In [43]:
english_s = []

In [44]:
for word in eng_words:
    english_s.append(f'{word}ing')

In [45]:
for word in eng_words:
    english_s.append(f'{word}s')

In [46]:
for word in eng_words:
    english_s.append(f'{word}es')

In [47]:
for word in eng_words:
    english_s.append(f'{word}ed')

In [48]:
eng_words = english_s + eng_words

In [49]:
len(eng_words)

31785

In [50]:
def predictFind_codemix(text): 
    
    global eng_words
    english_words = []
    text_for_pred = []
    for token in text.lower().split():
        if token in eng_words:
            english_words.append(token)
    
    text_for_pred = str([token for token in text.lower().split() if token not in english_words])
    
    global l, t, v
    x = t.transform([text_for_pred]).toarray() 
    lang = l.predict(x) 
    lang_per = l.predict_proba(x) 
    lang = v.inverse_transform(lang)
    print(f'''The text is in {str(lang[0])} with {round(float(lang_per.max()*100), 1)}% confidence \n\n''') 
    
    lang_names = list(v.inverse_transform(l.classes_)) 
    probas_list = []
    for i in lang_per:
        for j in i:
            probas_list.append(j)

    proba_table = pd.DataFrame({'Language': lang_names,
                        'Probability': probas_list}).sort_values(by='Probability',
                                                           ascending=False).reset_index(drop=True)
                
    if len(english_words)>0 and str(proba_table['Language'].loc[0])!='English':
        with open(f'''{proba_table['Language'].loc[0]}-English-codemix.csv''', 'a') as file:
            file.write(f'{text}\n')
            file.close()  
            
    elif len(english_words)>0 and str(proba_table['Language'].loc[0])=='English' and proba_table['Probability'].loc[0]<0.13:
        with open(f'''Unknown-dialect-codemix.csv''', 'a') as file:
            file.write(f'{text}\n')
            file.close()  
            
    elif len(english_words)>0 and str(proba_table['Language'].loc[0])=='English' and proba_table['Probability'].loc[0]>0.13:
        pass

In [ ]:
for index, file in enumerate(files):
    print(f'CSV file number: {index}')
    df = pd.read_csv(file)['sentence']
    df.drop_duplicated(inplace=True)
    for text in df.values:
        predictFind_codemix(text)